In [7]:
# make csv ready to datasets fromat
import pandas as pd

d_a = pd.read_csv("data/twitter_data/labels_tweets_dropna.csv")

from sklearn.model_selection import train_test_split
d_a['label'] = d_a['Cyberbullying'] + d_a['Insult'] + d_a['Threat']
d_a['label'] = d_a['label'].apply(lambda x: 1 if x > 0 else 0)

d_a_balanced = d_a[d_a.label == 0].iloc[:5017,:].append(d_a[d_a.label ==1]).sample(frac=1)
d_a_r = d_a_balanced[['Tweet','label']]

d_train, d_test = train_test_split(d_a_r, test_size=0.2, random_state=42)
d_test1, d_val = train_test_split(d_test, test_size=0.2, random_state=42)

d_train.to_csv("data/twitter_data/tweet_train.csv",index=False)
d_test1.to_csv("data/twitter_data/tweet_test.csv",index=False)
d_val.to_csv("data/twitter_data/tweet_val.csv",index=False)

In [11]:
from datasets import load_dataset

data_files = {"train": "data/twitter_data/tweet_train.csv", "test": "data/twitter_data/tweet_test.csv", "val": "data/twitter_data/tweet_val.csv"}
dataset = load_dataset("csv", data_files=data_files)


Using custom data configuration default-78a077952ce3eed9


Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

Dataset csv downloaded and prepared to C:/Users/bruger/.cache/huggingface/datasets/csv/default-78a077952ce3eed9/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [12]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")


def tokenize_function(examples):
    return tokenizer(examples["Tweet"], padding="max_length", truncation=True)


tokenized_datasets = dataset.map(tokenize_function, batched=True)

  0%|          | 0/9 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [13]:
small_train_dataset = tokenized_datasets["train"]
small_eval_dataset = tokenized_datasets["test"]

In [14]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=2)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [15]:
from transformers import TrainingArguments

training_args = TrainingArguments(output_dir="test_trainer")

In [16]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

In [17]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [18]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch")

In [19]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

In [20]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: Tweet. If Tweet are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
c:\Users\bruger\miniconda3\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 100
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 39
  Number of trainable parameters = 108311810


  0%|          | 0/39 [00:00<?, ?it/s]

The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: Tweet. If Tweet are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 100
  Batch size = 8


  0%|          | 0/13 [00:00<?, ?it/s]

{'eval_loss': 0.6887476444244385, 'eval_accuracy': 0.48, 'eval_runtime': 159.3794, 'eval_samples_per_second': 0.627, 'eval_steps_per_second': 0.082, 'epoch': 1.0}


KeyboardInterrupt: 